# Chainerの基礎

- 習得が簡単なインターフェイスで作られている
- 論文レベルでカスタマイズする際にとても柔軟に対応できる
- ChainerFamilyが充実してきている（ChainerRL，ChainerCV，ChainerMN）

## ChainerFamilyとは
- ChainerRL(Reinforcement learning)  
強化学習に特化したもの
- ChainerCV(Computer Vision)  
画像処理に特化したもの
- ChainerMN(Multi Node)  
並列処理に特化したもの



In [1]:
# 必要ライブラリのインストール
!pip install chainer
!pip install numpy

In [1]:
import chainer

D:\Anaconda3\envs\Chainer\lib\site-packages\chainer\backends\cuda.py:98: UserWarning: cuDNN is not enabled.
Please reinstall CuPy after you install cudnn
(see https://docs-cupy.chainer.org/en/stable/install.html#install-cudnn).
  'cuDNN is not enabled.\n'


 ## リンクの定義
 

In [2]:
import chainer.links as L

In [3]:
# 全結合層 (fuilly-connected)
fc = L.Linear(3, 2)

In [4]:
fc

In [5]:
# 上記のセルを実行時に重みがランダムに生成してくれている。
# 重みの確認
fc.W

variable W([[ 0.97804636, -0.8268776 ,  0.2865151 ],
            [-0.9042719 , -0.7814424 ,  1.3244693 ]])

In [7]:
# バイアスの確認（バイアスは小文字の『b』なので注意）
fc.b

variable b([0., 0.])

ランダムで生成される部分は、再現性が損なわれるため、気をつける必要がある。  
しっかりしていないと、精度の良いモデルができても、再実行すると、再現できない可能性がある。

In [8]:
import numpy as np

In [10]:
# numpyの乱数初期値で指定する
np.random.seed(3)
fc = L.Linear(3,2)

In [11]:
fc.W

variable W([[ 1.0326651 ,  0.25201908,  0.05571284],
            [-1.075888  , -0.16015016, -0.20482019]])

In [13]:
fc.b

variable b([0., 0.])

### なぜ、Chainerのシードを固定するために、Numpyのシードを固定するのか？
Chainerは、乱数を生成する際、Numpyを使用し、乱数を生成しているから！！

## 線形変換

In [18]:
# 入力する配列を定義する。
x = np.array([[1, 2, 3]])
x.dtype

dtype('int32')

In [17]:
# int float問題で、エラーが発生する。（Chainerの入力は、32Bitのfloat型）
u = fc(x)

InvalidType: 
Invalid operation is performed in: LinearFunction (Forward)

Expect: x.dtype.kind == f
Actual: i != f

In [19]:
x = np.array([[1, 2, 3]], 'f')
x.dtype

dtype('float32')

In [20]:
u = fc(x)

In [22]:
#  ３つの入力を入れ、中間層の２つの値を出力
u

variable([[ 1.7038419, -2.010649 ]])

## 非線形変換
relu関数を使用してみる。


In [23]:
import chainer.functions as F

In [24]:
z = F.relu(u)

In [25]:
z

variable([[1.7038419, 0.       ]])

### 問題設定
入力`[1, 2, 3]`に対する出力`y`の値を計算してください。

#### 構成
- 左側のリンク：fc1 → (3,2)
- 右側のリンク：fc2 → (2,1)
- fc1の非線形変換（Relu） → z1
- fc2の線形変換 → 出力y

`np.random.seed(3)`
#### 解答

In [26]:
np.random.seed(3)

fc1 = L.Linear(3,2)
fc2 = L.Linear(2,1)

x = np.array([[1, 2, 3]], 'f')

u1 = fc1(x)
z1 = F.relu(u1)
y = fc2(z1)

In [27]:
# 予測値
y

variable([[-0.09968679]])

## 損失関数の計算
教師データ今回は『３』にする。

In [28]:
# 教師データ
t = np.array([[3]], 'f')

In [29]:
t

array([[3.]], dtype=float32)

In [30]:
# 平均二乗誤差の計算
loss = F.mean_squared_error(t, y)
loss

variable(9.608059)